In [1]:
import vtk
import numpy as np
import plotly.graph_objs as go
from ipywidgets import widgets
from IPython.display import display
from plotly.subplots import make_subplots
from IPython.display import clear_output

# Load the data
reader = vtk.vtkXMLImageDataReader()
reader.SetFileName("mixture.vti")
reader.Update()
data = reader.GetOutput()

# Extract the scalar data for the vertices of the 3D dataset
scalar_data = reader.GetOutput().GetPointData().GetScalars()

# Get the point coordinates
# x, y, z = np.mgrid[0:data.GetDimensions()[0], 0:data.GetDimensions()[1], 0:data.GetDimensions()[2]]
x = []
y = []
z = []

#Fetch (x, y, z) coordinates for each point in order
for i in range(data.GetNumberOfPoints()):
    point = data.GetPoint(i)
    x.append(point[0])
    y.append(point[1])
    z.append(point[2])

# Convert the vtkFloatArray to a numpy array
values = np.array(data.GetPointData().GetScalars())

# Define the isovalue slider
slider = widgets.FloatSlider(
    value=0.0,
    min=np.min(values),
    max=np.max(values),
    step=0.01,
    description='Isovalue:',
    continuous_update=False
)

# Define the reset button
reset_button = widgets.Button(description='Reset', 
                              button_style='', 
                              style={'button_color': '#D3D3D3',
                                     'border-color': 'blue',
                                     'color': 'black'})




# Create the isosurface
fig = go.Figure(data=go.Isosurface(
    x=x,
    y=y,
    z=z,
    value=values.flatten(),
    isomin=0.0,
    isomax=0.0,
    cauto = False,
    cmin = np.min(values),
    cmax = np.max(values),
    surface_count=1,
    colorscale='plasma',
    caps=dict(x_show=False, y_show=False),
    opacity=1,
    colorbar=dict(thickness=20, ticklen=4),
    showscale=False,
))

# Set the camera position
fig.update_layout(scene_camera=dict(
    eye=dict(x=1.5, y=1.5, z=1.5),
    center=dict(x=1, y=1, z=1)
))

# Define the x-axis range of the histogram
x_min = np.min(values)
x_max = np.max(values) 

# Filter the scalar data to include only values within the x-axis range
filtered_scalar_data = [scalar_data.GetValue(i) for i in range(scalar_data.GetNumberOfTuples()) if x_min <= scalar_data.GetValue(i) <= x_max]

# Create a histogram plot
histogram = go.Figure(data=[go.Histogram(x=filtered_scalar_data, nbinsx=43,marker=dict(color='blue'),opacity=0.6)])



# Define the update function
def update(change):
    fig.data[0].isomin = slider.value
    fig.data[0].isomax = slider.value
    histogram.data[0].x = values.flatten()[(values.flatten() >= slider.value - 0.25) & (values.flatten() <= slider.value + 0.25)]
    
    fig.data[0].colorscale = 'plasma'
    fig.data[0].cauto = False
    fig.data[0].cmin = np.min(values)
    fig.data[0].cmax = np.max(values)
    fig.data[0].opacity=1

    clear_output(wait=True)
    display(widgets.HBox([slider, reset_button]))
    
   
    
    fig1 = make_subplots(rows=1, cols=2, specs=[[{'type': 'scene'}, {'type': 'xy'}]])
    fig1.add_trace(fig.data[0], row=1, col=1)
    fig1.add_trace(histogram.data[0], row=1, col=2)
    
    # Set the x-axis and y-axis labels for the histogram plot
    fig1.update_xaxes(title_text="Vortex scalar values", row=1, col=2)
    fig1.update_yaxes(title_text="Frequency", row=1, col=2)

    # Set the layout for the subplot
    fig1.update_layout(height=500, width=1000, title_text="Combined plot")

    # Show the plot
    fig1.show()

    
# Link the slider and the update function
slider.observe(update, 'value')


# Define the reset function
def reset(change):
    slider.value = 0.0
#     update(None)
    fig.data[0].isomin = slider.value
    fig.data[0].isomax = slider.value
    histogram.data[0].x = values.flatten()[(values.flatten() >= x_min) & (values.flatten() <= x_max )]

    fig.data[0].colorscale = 'plasma'
    fig.data[0].cauto = False
    fig.data[0].cmin = np.min(values)
    fig.data[0].cmax = np.max(values)
    fig.data[0].opacity=1

    clear_output(wait=True)
    display(widgets.HBox([slider, reset_button]))
    
   
    
    fig1 = make_subplots(rows=1, cols=2, specs=[[{'type': 'scene'}, {'type': 'xy'}]])
    fig1.add_trace(fig.data[0], row=1, col=1)
    fig1.add_trace(histogram.data[0], row=1, col=2)
    
    # Set the x-axis and y-axis labels for the histogram plot
    fig1.update_xaxes(title_text="Vortex scalar values", row=1, col=2)
    fig1.update_yaxes(title_text="Frequency", row=1, col=2)

    # Set the layout for the subplot
    fig1.update_layout(height=500, width=1000)

    # Show the plot
    fig1.show()

# Link the reset button and the reset function
reset_button.on_click(reset)

# Display the widgets and the figures
display(widgets.HBox([slider, reset_button]))

# Combine the two plots in a horizontal subplot
fig1 = make_subplots(rows=1, cols=2, specs=[[{'type': 'scene'}, {'type': 'xy'}]])
fig1.add_trace(fig.data[0], row=1, col=1)
fig1.add_trace(histogram.data[0], row=1, col=2)

# Set the x-axis and y-axis labels for the histogram plot
fig1.update_xaxes(title_text="Vortex scalar values", row=1, col=2)
fig1.update_yaxes(title_text="Frequency", row=1, col=2)

# Set the layout for the subplot
fig1.update_layout(height=500, width=1000)

# Show the plot
fig1.show()